# PREPROCESSING ORTHO TO LINKED DATA
In this notebook, we evaluate the Orthomosaics in a session repository.
For every tiff with its accompanying, an IMAGENODE metadata class is created 
that governs all the metadata of the image (paths, pose, etc.).
As ouput, the method generates RDF GRAPHS (.ttl) files

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [1]:
from rdflib import Graph, plugin
import uuid    
import importlib
import PIL.Image as PILimage
import os.path

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

## 1. INITIALIZE SESSION

In [3]:
## INPUTS
projectPath= os.path.abspath(os.path.join(os.getcwd(), os.pardir))+"\\tests"#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = projectPath + "\\Samples" #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
accuracy=0.1
gsd=0.01

## 2. READ PRESENT TiFF CONTAINING ORTHOMOSAIC
An IMAGENODE is created per image in the session

In [4]:
# Retrieve all image files in the session along with their xml metadata
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
nodelist=[]

for path in allSessionFilePaths:        
    if path.endswith(".tif"): 
        #create imgnode
        print('creating ImageNode from '+path)
        node=geomapi.imagenode.ImageNode()
        node.sessionPath=sessionPath
        node.timestamp=tl.get_timestamp(path) # this is a long float nr
        node.name=tl.get_filename(path)
        node.guid= '{'+str(uuid.uuid1())+'}'    
        node.path=path
        node.gsd=gsd
        node.accuracy=accuracy
        nodelist.append(node)              
print(str(len(nodelist))+' ImageNodes are created.')
print('Note that these nodes do not yet contain actual data but only the metadata')

creating ImageNode from d:\Scan-to-BIM repository\geomapi\tests\Samples\OrthoLAMBERT08_ELLIPS.tif
1 ImageNodes are created.
Note that these nodes do not yet contain actual data but only the metadata


In [5]:
#READ Data
for node in nodelist:
    node.img=PILimage.open(node.path)          

## 3. CREATE RDF GRAPH FOR All IMAGENODES AND EXPORT THEM TO .TTL
An RDF graph and .ttl file is created for all images in the session
(images themselves are not stored in the graph, only metadata)

In [6]:
graph=tl.nodes_to_graph(nodelist)
graph.serialize(destination=sessionPath+"\\orthoGraph.ttl", format='ttl')
print (str(len(nodelist))+' ImageNodes succesfully serialized in: '+sessionPath)
print(graph.serialize())

1 ImageNodes succesfully serialized in: d:\Scan-to-BIM repository\geomapi\tests\Samples
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix v4d: <https://w3id.org/v4d/core#> .

<http://OrthoLAMBERT08_ELLIPS> a "<class 'geomapi.imagenode.ImageNode'>" ;
    rdf:gsd "0.01" ;
    v4d:accuracy "0.1" ;
    v4d:guid "{e3b043a3-bb3c-11ec-9c46-c8f75043ce59}" ;
    v4d:path "OrthoLAMBERT08_ELLIPS.tif" ;
    openlabel:timestamp "2022-04-07 08:55:45" .


